In [11]:

class MysqlConn(object):
    def __init__(self, config_json_str):
        # self.mysql_config = {"host": "10.22.128.150",  "port": 3317, "db": "bigdata", "user": "talend_load",
        #   "passwd": "s9t5gNThn2vqWM7c" , "charset" : "utf8"}
        # self.mysql_config = {"host": "db-bigdata.wmcloud-qa.com",  "port": 3312, "db": "bigdata", "user": "app_bigdata_ro",
        #   "passwd": "Welcome_20141217"}
        self.mysql_config = json.loads(config_json_str, encoding='utf-8')

    def connect(self):
        mysql_conn = pymysql.connect(**self.mysql_config)
        return mysql_conn

class MssqlConn(object):
    def __init__(self, config_json_str):
        # self.mssql_config = {"server": "sh-datayesdb.wmcloud-dev.com",  "port": 1433, "database": "datayesdb",
        #     "user": "talend_load", "password": "Welcome01"}
        self.mssql_config = json.loads(config_json_str, encoding='utf-8')

    def connect(self):
        mssql_conn = pymssql.connect(**self.mssql_config)
        return mssql_conn

    def __str__(self):
        return json.dumps(self.mssql_config, ensure_ascii=False, encoding='utf-8')



In [33]:
import json
import pymysql
import pymssql
# MySQL
bigdata={"host":"security03-dev.datayes.com","port":3306,"user":"talend_load","passwd":"NCph1G9BQT3DuQj","db":"bigdata","charset":"utf8"}
dateyesdbp={"host":"db-datayesdb-ro.wmcloud.com","port":3313,"user":"app_dataqa_ro","passwd":"Welcome20140820","db":"datayesdbp","charset":"utf8"}

# dateyesdbp={"host":"db-datayesdb.wmcloud.com","port":3312,"user":"app_marketdata","passwd":"JDFJ8dfasd8aKfu","db":"datayesdbp","charset":"utf8"}

# SQLserver
datayes={"server":"sh-dm-db05.datayes.com","port":1433,"database":"dyedb","user":"talend_load","password":"s9t5gNThn2vqWM7c","charset":"utf8"}


bigdata_conn = MysqlConn(json.dumps(bigdata)).connect()
dateyesdbp_conn = MysqlConn(json.dumps(dateyesdbp)).connect()

datayes_conn = MssqlConn(json.dumps(datayes)).connect()


In [ ]:
# 从dyedy MSsql库中查询party_id

In [16]:
import pandas as pd

# to get party_id list
sql = """
select party_id from md_security where EXCHANGE_CD in ('XSHG', 'XSHE') and ASSET_CLASS='E' and party_id in (
select distinct(party_id) from md_inst_type where type_id in 
(
select type_id from md_type where type_name like '%计算机%'
)
)
"""
party_ids = pd.read_sql(sql, datayes_conn)

print party_ids.head()

   party_id
0        69
1      1251
2       361
3    568715
4       507


In [38]:
# party_ids.T.iloc[0].values
party_ids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494 entries, 0 to 493
Data columns (total 1 columns):
party_id    494 non-null int64
dtypes: int64(1)
memory usage: 3.9 KB


In [27]:
# 从datayesdbp mssql库中读取财务数据

In [36]:
sql = """
select * from vfdmt_indi_ps_pit
where party_id in (%s)
""" % (','.join(party_ids.astype(str).T.iloc[0].values))
v = pd.read_sql(sql, dateyesdbp_conn)

print v.head()

   SECURITY_ID_INT  SECURITY_ID  PARTY_ID TICKER_SYMBOL SEC_SHORT_NAME  \
0               27  000014.XSHE        15        000014           沙河股份   
1               27  000014.XSHE        15        000014           沙河股份   
2               27  000014.XSHE        15        000014           沙河股份   
3               27  000014.XSHE        15        000014           沙河股份   
4               27  000014.XSHE        15        000014           沙河股份   

  SEC_SHORT_NAME_EN EXCHANGE_CD         ACT_PUBTIME PUBLISH_DATE    END_DATE  \
0              None        XSHE 2003-04-22 18:00:00   2003-04-23  2003-03-31   
1              None        XSHE 2003-07-16 18:00:00   2003-07-17  2003-06-30   
2              None        XSHE 2003-10-22 18:00:00   2003-10-23  2003-09-30   
3              None        XSHE 2004-03-29 18:00:00   2004-03-30  2003-12-31   
4              None        XSHE 2005-04-22 18:00:00   2005-04-23  2003-12-31   

          ...          C_RESER_PS  S_RESER_PS  RESER_PS   RE_PS  T_RE_PS  

In [37]:
v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38030 entries, 0 to 38029
Data columns (total 28 columns):
SECURITY_ID_INT      38030 non-null int64
SECURITY_ID          38030 non-null object
PARTY_ID             38030 non-null int64
TICKER_SYMBOL        38030 non-null object
SEC_SHORT_NAME       38030 non-null object
SEC_SHORT_NAME_EN    33002 non-null object
EXCHANGE_CD          38030 non-null object
ACT_PUBTIME          38030 non-null datetime64[ns]
PUBLISH_DATE         38030 non-null object
END_DATE             38030 non-null object
EPS                  36478 non-null float64
BASIC_EPS            31774 non-null float64
DILUTED_EPS          31663 non-null float64
N_ASSET_PS           24383 non-null float64
T_REV_PS             36431 non-null float64
REV_PS               36442 non-null float64
OP_PS                36458 non-null float64
EBIT_PS              23646 non-null float64
C_RESER_PS           23441 non-null float64
S_RESER_PS           23251 non-null float64
RESER_PS       

In [ ]:
# 获取过去一年股票增长率的情况， 分类， 筛选表现较好的股票， 分析它们的公司情况
# 逆推，不是用公司营业收入等等去推算未来股份的表现，而是倒推